In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# Calculates OPR/DPR/CCWM
# For implementation details, see
# https://www.chiefdelphi.com/forums/showpost.php?p=484220&postcount=19

# M is n x n where n is # of teams
# s is n x 1 where n is # of teams
# solve [M][x]=[s] for [x]

# x is OPR and should be n x 1

# Match format:
# match_number, red1, red2, red3, blue1, blue2, blue3, redscore, bluescore

## Event code is set in line 112ish

import numpy as np
import requests
import csv
import pandas as pd

baseURL = 'http://www.thebluealliance.com/api/v3/'
#baseURL = 'http://127.0.0.1:8080/'
header = {'X-TBA-Auth-Key':'2dZKrdov7pGW7HW3kA7Ib8MXa6je84N8ecaqhk3dbFK0SACi27TS8XLU4P5U6GPf'} 

ses = requests.Session()

def getTBA(url):
	return ses.get(baseURL + url, headers=header).json()


numTeams = 3

red_index = [1, numTeams+1]
blue_index = [numTeams+1, (numTeams*2)+1]


def build_team_mapping(matches):
    # Build team list

    team_list = set()

    #any team key is added.
    for match in matches:
        for team in match[1:(numTeams*2)+1]:
            team_list.add(team)

    #Converted to list to make it unique keys.
    team_list = list(team_list)
    team_id_map = {}
    for (i, team) in enumerate(team_list):
        team_id_map[team] = i

    return (team_list, team_id_map)


def build_Minv_matrix(matches, team_id_map):
    n = len(team_id_map.keys())
    M = np.zeros([n, n]) #n x n array of teams to solve.
    for match in matches:
        for alliance_color in [red_index, blue_index]:
            alliance_teams = match[alliance_color[0]:alliance_color[1]]
            for team1 in alliance_teams:
                team1_id = team_id_map[team1]
                for team2 in alliance_teams:
                    M[team1_id, team_id_map[team2]] += 1
    return np.linalg.pinv(M)

def build_s_matrix(matches,team_id_map):
    n = len(team_id_map.keys())
    s = np.zeros([n, 1])
    for match in matches:
		#We used to ignore non-qf matches here.
		#instead do it on input to reduce dimensionality or something.
        for alliance_color in [red_index, blue_index]:
            alliance_teams = [team for team in
                              match[alliance_color[0]:alliance_color[1]]]
            stat = _get_stat(match, alliance_color)

            for team in alliance_teams:
                s[team_id_map[team]] = s[team_id_map[team]] + stat
    return s


def calc_stat(matches, team_list, team_id_map, Minv):
    import numpy as np

    s = build_s_matrix(matches, team_id_map) 
    x = np.dot(Minv, s)

    stat_dict = {}
    for (team, stat) in zip(team_list, x):
        stat_dict[team] = stat[0]
    return stat_dict


def _get_stat( match, alliance_color):
    if alliance_color == red_index:
        return match[7]
    if alliance_color == blue_index:
        return match[8]


def calculate_matchstats(matches):
    (team_list, team_id_map) = build_team_mapping(matches)
    Minv = build_Minv_matrix(matches, team_id_map)
    oprs_dict = calc_stat(matches, team_list, team_id_map, Minv)
    return oprs_dict


eventCode = "2022wasno"# Set event code

eventMatches = [x for x in getTBA("event/" + eventCode + "/matches") if x['comp_level'] == "qm" and x['actual_time'] != None]


keysWeCanUse = []
for key in eventMatches[0]['score_breakdown']['red'].keys():
	if type(eventMatches[0]['score_breakdown']['red'][key]) != type(""):
		keysWeCanUse.append(key)

print(keysWeCanUse)
for cOPRKey in keysWeCanUse:
	currentMatchData = []
	for match in eventMatches:
		if 'score_breakdown' in match:
			#print((match['alliances']['red']['team_keys']))
			currentMatchData.append([match['key']] + match['alliances']['red']['team_keys'] + match['alliances']['blue']['team_keys'] + [match['score_breakdown']['red'][cOPRKey]] + [match['score_breakdown']['blue'][cOPRKey]])
			#print(currentMatchData[-1])


	x = calculate_matchstats(currentMatchData)
	x = sorted([[y,x[y]] for y in x], key=lambda opr:opr[1], reverse=True)
	#print(x)
	with open((eventCode + "_" + cOPRKey + ".csv"), 'w') as csvfile:
		sw = csv.writer(csvfile, delimiter=',',
                            quotechar='|')
		for dataItem in x:
			#print(dataItem)
			sw.writerow(dataItem)



['adjustPoints', 'autoCargoLowerBlue', 'autoCargoLowerFar', 'autoCargoLowerNear', 'autoCargoLowerRed', 'autoCargoPoints', 'autoCargoTotal', 'autoCargoUpperBlue', 'autoCargoUpperFar', 'autoCargoUpperNear', 'autoCargoUpperRed', 'autoPoints', 'autoTaxiPoints', 'cargoBonusRankingPoint', 'endgamePoints', 'foulCount', 'foulPoints', 'hangarBonusRankingPoint', 'matchCargoTotal', 'quintetAchieved', 'rp', 'techFoulCount', 'teleopCargoLowerBlue', 'teleopCargoLowerFar', 'teleopCargoLowerNear', 'teleopCargoLowerRed', 'teleopCargoPoints', 'teleopCargoTotal', 'teleopCargoUpperBlue', 'teleopCargoUpperFar', 'teleopCargoUpperNear', 'teleopCargoUpperRed', 'teleopPoints', 'totalPoints']


In [4]:
import pandas as pd
import numpy as np
import math
import time
import sys
from IPython.display import display 
import csv
from astropy import units as u
from astropy.coordinates import Angle

#still needs to be cleaned up, but should work
pd.set_option("display.max_rows", None, "display.max_columns", None)


OPRt = pd.read_csv ('2022wasno_matchCargoTotal.csv',index_col=None, header=None ) 

type(OPRt)
OPRt.columns = ['Team', 'Cargo Count OPR']
display(OPRt)
OPR8248wasno = OPRt[OPRt['Team'].str.contains('8248')]
print (OPR8248wasno)

OPRt = pd.read_csv ('2022wabon_matchCargoTotal.csv',index_col=None, header=None ) 

type(OPRt)
OPRt.columns = ['Team', 'Cargo Count OPR']
display(OPRt)
OPR8248wabon = OPRt[OPRt['Team'].str.contains('8248')]
print (OPR8248wabon)

OPRt = pd.read_csv ('2022pncmp_matchCargoTotal.csv',index_col=None, header=None ) 

type(OPRt)
OPRt.columns = ['Team', 'Cargo Count OPR']
display(OPRt)
OPR8248wacmp = OPRt[OPRt['Team'].str.contains('8248')]
print (OPR8248wacmp)

,Team,Cargo Count OPR
0,frc2910,16.969805
1,frc2930,12.312802
2,frc4089,10.314658
3,frc7461,10.063710
4,frc5827,9.823547
5,frc4131,9.585872
6,frc2980,8.952804
7,frc492,6.641437
8,frc1778,6.081680
9,frc2522,6.060575


       Team  Cargo Count OPR
17  frc8248         3.377311


,Team,Cargo Count OPR
0,frc2910,20.599374
1,frc2046,15.301521
2,frc4469,11.442710
3,frc2557,11.117233
4,frc2976,10.977051
5,frc3826,9.269502
6,frc3218,9.125269
7,frc8248,7.949010
8,frc3070,7.402551
9,frc1294,6.233180


      Team  Cargo Count OPR
7  frc8248          7.94901


,Team,Cargo Count OPR
0,frc2910,19.607102
1,frc4911,16.869277
2,frc2046,15.049567
3,frc2471,14.374558
4,frc2147,13.924381
5,frc5827,13.712653
6,frc360,13.567028
7,frc4488,13.349438
8,frc955,12.834893
9,frc2976,12.579451


       Team  Cargo Count OPR
42  frc8248         5.194711


In [5]:
mainSeason = pd.concat([OPR8248wasno, OPR8248wabon])
fullSeason = pd.concat([mainSeason, OPR8248wacmp])
#print(fullSeason)

df = pd.DataFrame(data=fullSeason)

print(df)

Total = df.at[17, 'Cargo Count OPR']+df.at[7, 'Cargo Count OPR']+df.at[42, 'Cargo Count OPR']
final=(Total/3)/2
print("Total avg cycles by OPR=", final)
print("WASNO avg cycles by OPR=", (df.at[17, 'Cargo Count OPR']/2))
print("WABON avg cycles by OPR=", (df.at[7, 'Cargo Count OPR']/2))
print("PNWCMP avg cycles by OPR=", (df.at[42, 'Cargo Count OPR']/2))


       Team  Cargo Count OPR
17  frc8248         3.377311
7   frc8248         7.949010
42  frc8248         5.194711
Total avg cycles by OPR= 2.753505519710752
WASNO avg cycles by OPR= 1.6886556550335956
WABON avg cycles by OPR= 3.9745052423258875
PNWCMP avg cycles by OPR= 2.597355661772773
